In [66]:
from transformers import BertTokenizer, BertForTokenClassification, AdamW
from transformers import BertTokenizerFast
import torch
import os
import re
from tqdm import tqdm
#from seqeval.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset

In [67]:
# Extract sentences from the dataset
oraciones_entrenamiento = [[token for token, tag in oracion] for oracion in dataset_entrenamiento]


In [68]:
def preprocesar_texto(texto):
    texto_limpio = re.sub(r"\s+", " ", texto).strip()  # Elimina espacios extra
    texto_limpio = texto_limpio.replace('\n', ' ') #elimina saltos de linea
    return texto_limpio

In [69]:

# Initialize the tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

def tokenizar_y_codificar(oraciones, tokenizer, etiquetas):

    tokens_codificados = tokenizer(oraciones, is_split_into_words=True, 
                                    return_tensors="pt", padding=True, truncation=True)
    etiquetas_codificadas = []
    for i, oracion in enumerate(oraciones):
        etiquetas_oracion = []
        for j in range(len(oracion)):
            _, etiqueta = oracion[j]
            etiquetas_oracion.append(etiquetas[etiqueta])

        # Añadir etiquetas para [CLS] y [SEP]
        etiquetas_oracion.insert(0, etiquetas["O"])  # [CLS]
        etiquetas_oracion.append(etiquetas["O"])  # [SEP]


        etiquetas_codificadas.append(torch.tensor(etiquetas_oracion))

    return tokens_codificados, etiquetas_codificadas


# Etiquetas
etiquetas = {"O": 0, "FECHA": 1, "DIRECTOR": 2, "NUM_PAGINAS": 3}
num_etiquetas = len(etiquetas)

# Modelo
model = BertForTokenClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", num_labels=num_etiquetas)
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")


# Dataset de entrenamiento (ejemplo) - ¡DEBES REEMPLAZAR ESTO CON TUS DATOS!
dataset_entrenamiento = [
    [("Bogotá,", "O"), ("D.","O"), ("C.,", "O"), ("jueves,", "O"), ("9", "FECHA"), ("de", "FECHA"), ("febrero", "FECHA"), ("de", "FECHA"), ("2023", "FECHA")],
    [("DIRECTORES:", "O"), ("GREGORIO", "DIRECTOR"), ("ELJACH", "DIRECTOR"), ("PACHECO", "DIRECTOR")],
    [("EDICIÓN", "O"), ("DE", "O"), ("22", "NUM_PAGINAS"), ("PÁGINAS", "O")]
]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [72]:
from transformers import BertTokenizerFast
import torch

def tokenizar_codify(oraciones, dataset_entrenamiento, tokenizer, tags):
    tokens_codified = tokenizer(
        oraciones,
        is_split_into_words=True,  # True if the input is already tokenized
        return_tensors="pt",
        padding=True,
        truncation=True,
    )
    
    # Align the tags with the tokenized input
    tags_codified_list = []
    for oracion in dataset_entrenamiento:
        tags_list = [tags[tag] for token, tag in oracion]
        word_ids = tokens_codified.word_ids()
        tags_codified = []
        for word_id in word_ids:
            if word_id is None:
                tags_codified.append(-100)  # Special token
            else:
                tags_codified.append(tags_list[word_id])
        tags_codified_list.append(tags_codified)
    
    tags_codified = torch.tensor(tags_codified_list)
    return tokens_codified, tags_codified

# Example dataset_entrenamiento
dataset_entrenamiento = [
    [("Bogotá,", "O"), ("D.","O"), ("C.,", "O"), ("jueves,", "O"), ("9", "FECHA"), ("de", "FECHA"), ("febrero", "FECHA"), ("de", "FECHA"), ("2023", "FECHA")],
    [("DIRECTORES:", "O"), ("GREGORIO", "DIRECTOR"), ("ELJACH", "DIRECTOR"), ("PACHECO", "DIRECTOR")],
    [("EDICIÓN", "O"), ("DE", "O"), ("22", "NUM_PAGINAS"), ("PÁGINAS", "O")]
]

# Extract sentences from the dataset
oraciones_entrenamiento = [[token for token, tag in oracion] for oracion in dataset_entrenamiento]

# Example tags dictionary
etiquetas = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-LOC": 3,
    "I-LOC": 4,
    "B-ORG": 5,
    "I-ORG": 6,
    "B-MISC": 7,
    "I-MISC": 8,
    "B-FECHA": 9,  # Add 'FECHA' tag
    "I-FECHA": 10  # Add 'FECHA' tag
}

# Initialize the tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Tokenize and codify the data
tokens_codified, tags_codified = tokenizar_codify(oraciones_entrenamiento, dataset_entrenamiento, tokenizer, etiquetas)

# Debug prints to check lengths
print(f"Length of input_ids: {len(tokens_codified['input_ids'][0])}")
print(f"Length of attention_mask: {len(tokens_codified['attention_mask'][0])}")
print(f"Length of tags_codified: {len(tags_codified[0])}")

KeyError: 'FECHA'

## Train Data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) #mover el modelo al dispositivo

learning_rate = 5e-5  # Ajusta según tus necesidades
optimizer = AdamW(model.parameters(), lr=learning_rate)


num_epochs = 3  # Ajusta según tus necesidades

for epoch in range(num_epochs):
    total_loss = 0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False)  # leave=False para barra de progreso limpia
    for batch in progress_bar:  # Usa una barra de progreso
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad() #limpiar gradientes
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward() #calculo de gradientes
        optimizer.step() #actualizacion de los pesos
        progress_bar.set_postfix({"Loss": loss.item()}) #se imprime la perdida
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {avg_loss:.4f}")

c:\Users\Jorge\anaconda3\envs\Extract\Lib\site-packages\transformers\optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


NameError: name 'dataloader' is not defined

# Load  model

In [ ]:
nombre_modelo = "mi_modelo_bert_gacetas"  # O el nombre que prefieras
ruta_modelo = os.path.join("modelos", nombre_modelo)
os.makedirs(ruta_modelo, exist_ok=True) #crea las carpetas si no existen
model.save_pretrained(ruta_modelo)  # Guarda el modelo
tokenizer.save_pretrained(ruta_modelo)  # Guarda el tokenizer

print(f"Modelo guardado en: {ruta_modelo}")

## Evolucion

In [ ]:
from transformers import pipeline

# Cargar el modelo entrenado
model = BertForTokenClassification.from_pretrained(ruta_modelo)
tokenizer = BertTokenizer.from_pretrained(ruta_modelo)
nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0 if torch.cuda.is_available() else -1)

# Ejemplo de dataset de prueba (¡DEBES REEMPLAZAR ESTO CON TUS DATOS!)
dataset_prueba = [
    [("Bogotá,", "O"), ("D.", "O"), ("C.,", "O"), ("miercoles,", "O"), ("15", "FECHA"), ("de", "FECHA"), ("marzo", "FECHA"), ("de", "FECHA"), ("2023", "FECHA")],
    [("DIRECTORES:", "O"), ("MARIA", "DIRECTOR"), ("JOSE", "DIRECTOR"), ("SUAREZ", "DIRECTOR")],
    [("EDICIÓN", "O"), ("DE", "O"), ("30", "NUM_PAGINAS"), ("PÁGINAS", "O")]
]

oraciones_prueba = [oracion for oracion in dataset_prueba] #se obtienen las oraciones
verdaderas_etiquetas = [[etiqueta for _, etiqueta in oracion] for oracion in dataset_prueba] #se obtienen las etiquetas

predicciones = []
for oracion in oraciones_prueba:
    oracion_texto = " ".join([token for token, _ in oracion]) # se unen los tokens
    oracion_preprocesada = preprocesar_texto(oracion_texto) #se preprocesa la oracion
    entidades = nlp(oracion_preprocesada)
    etiquetas_predichas = obtener_etiquetas_predichas(entidades, oracion, etiquetas) # se obtienen las etiquetas predichas
    predicciones.append(etiquetas_predichas) # se agregan a la lista de predicciones

reporte = classification_report(verdaderas_etiquetas, predicciones) #se genera el reporte
print(reporte)

def obtener_etiquetas_predichas(entidades, oracion_original, etiquetas): #funcion para obtener las etiquetas predichas
    etiquetas_predichas = ["O"] * len(oracion_original) #se inicializa la lista de etiquetas predichas con la etiqueta "O"

    for entidad in entidades:
        inicio = entidad["start"]
        fin = entidad["end"]
        etiqueta_predicha = entidad["entity_group"]

        for i in range(len(oracion_original)): #itera sobre los tokens de la oracion original
            token_inicio = oracion_original[i][0]

            if oracion_original[i][0] in entidad["word"]:
                 etiquetas_predichas[i] = etiqueta_predicha

    return etiquetas_predichas